In [30]:
import pandas as pd


targetGenes = ['TBC1D9', 'GATA3', 'SLC16A6', 'ESR1', 'INPP4B', 'SLC44A4', 'ANXA9', 'AGR2', 'MCCC2', 'TSPAN1', 'STBD1', 'MLPH', 'CACNA2D2',
'RARA', 'STARD3', 'PPP1R14C', 'SFRS13B', 'LDHB', 'MFGE8', 'PSAT1']

In [31]:
df = pd.read_csv('../Data/clinical.csv')

In [32]:
geneFiles = df['file']

In [ ]:
dfPatientGenes = pd.DataFrame()

for idx, file_name in enumerate(geneFiles, start=1):
    file_path = f"../Data/{file_name}"
    
    dfGenes = pd.read_csv(file_path, sep='\t', skiprows=[0])
    
    # Columns may have been dropped if first line in file were columns, load data again without dropping first line
    if "gene_name" not in dfGenes.columns:
        dfGenes = pd.read_csv(file_path, sep='\t')
    
    #dfGenes
    
    print(f"File {idx} - {file_name}", end="\r")
    
    #Gather all targeted genes we want with the values we want
    dfTarget = dfGenes[dfGenes['gene_name'].isin(targetGenes)][['gene_name', 'stranded_first']]

    #Transform the dfTarget to a single row dataframe with all genes as columns
    dfNewGenes = dfTarget.set_index('gene_name').T.reset_index(drop=True)

    #Add the patient uuid for reference and set it as the index
    patientID = df['bcr_patient_uuid'].loc[df['file'] == file_name]
    dfNewGenes['patient_uuid'] = patientID.values[0]
    dfNewGenes.set_index('patient_uuid', drop=True, inplace=True)
    
    #Concat the data to the final dataframe before saving it to a file
    dfPatientGenes = pd.concat([dfPatientGenes, dfNewGenes])

#For visualizing the result
#dfPatientGenes    

In [ ]:
#Send data to a csv

dfPatientGenes.to_csv('../Data/patient_genes.csv', index=False)